# Convolutional RBM (cRBM)

This notebook takes care of implementing the basic functionality for cRBMs.
Or maybe it's just for the preliminaries, that is some simple stuff before it actually comes to the Boltzmann Machine.


## Part 1: Initializing the weight matrices and applying them to sequences

In [196]:
#import theano
import numpy as np
import Bio.SeqIO as sio
import Bio.motifs.matrix as mat
from Bio.Alphabet import IUPAC

import random
import time

In [195]:
class FASTAReader:
    
    def __init__(self, _path):
        self.path = _path
        
    def readSequencesFromFile (self, filename):
        dhsSequences = []
        for dhs in sio.parse(open(filename), 'fasta', IUPAC.unambiguous_dna):
            dhsSequences.append(dhs)
        return dhsSequences
    

In [191]:
# apply the two classes to calculate a forward pass in our algorithm
seqReader = FASTAReader('.')
allSeqs = seqReader.readSequencesFromFile('../data/wgEncodeAwgDnaseUwAg10803UniPk.fa')


In [200]:
class ConvRBM:
    
    def __init__ (self, _motifLength, _numMotifs, _alphabet=IUPAC.unambiguous_dna):
        self.motifLength = _motifLength
        self.numMotifs = _numMotifs
        self.motifs = []
        self.alphabet = _alphabet
        print self.numMotifs
        self.bias = np.random.rand(self.numMotifs)
        self.c = random.random()
        
    def initializePWMs (self):
        # set up PWMs
        for m in range(self.numMotifs):
            self.motifs.append(self._createRandomMotif(self.motifLength, self.alphabet))
        
    def forwardPass (self, seq):
        if self.motifs == []:
            print 'Error: No motifs created so far. Try executing initializePWMs before!'
            return

        hiddenActivation = np.zeros((self.numMotifs, len(seq)-self.motifLength+1))
        motifCount = 0
        for motif in self.motifs:
            pssm = motif.log_odds()
            hiddenActivation[motifCount,:] = self._sigmoid(pssm.calculate(seq) + self.bias[motifCount])
            motifCount += 1

        return hiddenActivation
    
    def backwardPass (self, hiddenActivation):
        visibleActivation = np.zeros((1, hiddenActivation.shape[1]+self.motifLength-1))
        print visibleActivation.shape
        
        for motif in range(visibleActivation.shape[1]):
            pssm = self.motifs[motif].log_odds()
            visibleActivation = visibleActivation + pssm.calculate(hiddenActivation[motif,:]) + self.c
            
        return self._sigmoid(visibleActivation)
    
    def _createRandomMotif (self, motifLength, alphabet):
        counts = {}
        for letter in alphabet.letters:
            counts[letter] = [random.randint(0,100) for x in xrange(motifLength)]
        return mat.PositionWeightMatrix(alphabet, counts)
        
    def _sigmoid (self, x):
        return 1.0 / (1.0 + np.exp(-x))

In [201]:
learner = ConvRBM(6, 5)
learner.initializePWMs()

x = learner.forwardPass(allSeqs[246].seq)
y = learner.backwardPass(x)

5
(1, 150)


AttributeError: 'numpy.ndarray' object has no attribute 'alphabet'

In [190]:
start = time.time()
i = 0
lengthes = []
someScores = []
for seq in allSeqs:
    convoluted = learner.forwardPass(seq.seq)
    lengthes.append(len(seq))
    if i % 5000 == 0:
        someScores.append(convoluted[0][random.randint(0, len(convoluted))])
        print str(i) + " -> " + str(someScores[-1])
    i += 1
    
print
print
print "Number of filters: " + str(learner.numMotifs)
print "Number of DHSs: " + str(i)
print "Average Length of Sequences: " + str(np.array(lengthes).mean())
print "Execution Time: " + str(time.time()-start)

5
0 -> 0.906600534916
5000 -> 0.208888828754
10000 -> 0.0
15000 -> 0.0
20000 -> 0.0
25000 -> 0.641809999943
30000 -> 0.0
35000 -> 0.658836007118
40000 -> 0.57325977087
45000 -> 0.582175374031
50000 -> 0.0
55000 -> 0.551681339741
60000 -> 0.910535812378
65000 -> 0.624538183212
70000 -> 0.606583654881
75000 -> 0.767180681229
80000 -> 0.271857708693
85000 -> 0.474154680967
90000 -> 0.0
95000 -> 0.48353984952
100000 -> 0.0
105000 -> 0.291799157858
110000 -> 0.591895580292
115000 -> 0.0
120000 -> 0.952310502529
125000 -> 0.181974470615
130000 -> 0.684669852257
135000 -> 0.493779927492
140000 -> 0.771480441093
145000 -> 0.135087832808
150000 -> 0.0
155000 -> 0.0830420479178
160000 -> 0.214970812201
165000 -> 0.0
170000 -> 0.785442829132


Number of filters: 5
Number of DHSs: 171275
Average Length of Sequences: 150.0
Execution Time: 125.519943953


Some tests to learn how to do things with Biopython and Theano
===

### Do all DHS sequences have the same length by default?


In [51]:
fasta_seqs = sio.parse(open('../data/wgEncodeAwgDnaseUwAg10803UniPk.fa'), 'fasta')
count = 0
countNotSameLength = 0
for seq in fasta_seqs:
    if len(seq) != 150:
        print 'not length 150'
        countNotSameLength += 1
    count += 1

print 'Number of sequences: ' + str(count)
print 'Number of seqs with length != 150: ' + str(countNotSameLength)

Number of sequences: 171275
Number of seqs with length != 150: 0


In [52]:
import Bio.NeuralNetwork.Gene.Schema as schema
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC

### How do we generate random motif matrices (PWMs or PSSMs)

In [145]:
alphabet = IUPAC.unambiguous_dna
generator = schema.RandomMotifGenerator(alphabet, 6, 10)
for i in range(3):
    x = generator.random_motif()
    print x

CTGCGAT
CGTGACCA
GTACACCGA


In [54]:
from Bio import motifs

In [55]:
x = motifs.Motif(alphabet)
len(x)

0

In [56]:
import Bio.motifs.matrix as mat
import random

In [64]:
def createRandomMotif (motifLength, alphabet):
    counts = {}
    for letter in alphabet.letters:
        counts[letter] = [random.randint(0,100) for x in xrange(motifLength)]
    return mat.PositionWeightMatrix(alphabet, counts)
#x = mat.PositionWeightMatrix(alphabet, counts)

In [143]:
x = createRandomMotif(10, alphabet)
print x

        0      1      2      3      4      5      6      7      8      9
A:   0.28   0.05   0.38   0.08   0.08   0.29   0.40   0.20   0.06   0.07
C:   0.23   0.08   0.42   0.33   0.34   0.10   0.15   0.40   0.25   0.26
G:   0.24   0.20   0.18   0.24   0.18   0.42   0.16   0.18   0.35   0.20
T:   0.24   0.66   0.02   0.35   0.40   0.20   0.29   0.22   0.33   0.47



### Are the elements of a PWM interpretable as probabilites?

In [144]:
# verify that we're dealing with probabilities by summing up over all letters for each position
for pos in range(x.length):
    c = 0
    for letter in alphabet.letters:
        c += x[letter][pos]
    print str(pos) + " -> " + str(c)

0 -> 1.0
1 -> 1.0
2 -> 1.0
3 -> 1.0
4 -> 1.0
5 -> 1.0
6 -> 1.0
7 -> 1.0
8 -> 1.0
9 -> 1.0
